### W23P1 STAT 857 - Feature Engineering

In [ ]:
pip install haversine

In [1]:
## Importing libraries
import calendar
import numpy as np
import pandas as pd
from datetime import datetime
from haversine import haversine, Unit
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_columns', None, 'display.max_rows', None)

In [2]:
## Reading the data
train = pd.read_csv('Data/W23P1_train_google.csv')
test = pd.read_csv('Data/W23P1_test_google.csv')

In [3]:
## Defining helper dictionaries
nyc_airports = {'JFK':{'min_lng':-73.8352, 'min_lat':40.6195, 'max_lng':-73.7401, 'max_lat':40.6659}, 
                'EWR':{'min_lng':-74.1925, 'min_lat':40.6700,  'max_lng':-74.1531,  'max_lat':40.7081}, 
                'LGA':{'min_lng':-73.8895, 'min_lat':40.7664, 'max_lng':-73.8550, 'max_lat':40.7931}
               }

nyc_boroughs = {'manhattan':{'min_lng':-74.0479, 'min_lat':40.6829, 'max_lng':-73.9067, 'max_lat':40.8820}, 
                'queens':{'min_lng':-73.9630, 'min_lat':40.5431, 'max_lng':-73.7004, 'max_lat':40.8007}, 
                'brooklyn':{'min_lng':-74.0421, 'min_lat':40.5707, 'max_lng':-73.8334, 'max_lat':40.7395}, 
                'bronx':{'min_lng':-73.9339, 'min_lat':40.7855, 'max_lng':-73.7654, 'max_lat':40.9176}, 
                'staten_island':{'min_lng':-74.2558, 'min_lat':40.4960, 'max_lng':-74.0522, 'max_lat':40.6490}
               }

## Defining helper functions
def isAirport(latitude, longitude, airport_name = 'JFK'):
    
    a = (latitude >= nyc_airports[airport_name]['min_lat'])
    b = (latitude <= nyc_airports[airport_name]['max_lat'])
    c = (longitude >= nyc_airports[airport_name]['min_lng'])
    d = (longitude <= nyc_airports[airport_name]['max_lng'])
    
    if (a and b and c and d): return 1
    else: return 0

def getBorough(latitude, longitude):
    
    boroughs = nyc_boroughs.keys()
    
    for borough in boroughs:
        
        a = (latitude >= nyc_boroughs[borough]['min_lat'])
        b = (latitude <= nyc_boroughs[borough]['max_lat'])
        c = (longitude >= nyc_boroughs[borough]['min_lng'])
        d = (longitude <= nyc_boroughs[borough]['max_lng'])
        
        if (a and b and c and d): return borough
    
    return 'other'

In [4]:
## Date/time
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'], format = '%Y-%m-%d %H:%M:%S UTC')
train['pickup_date'] = train['pickup_datetime'].dt.date
train['pickup_day'] = train['pickup_datetime'].apply(lambda x:x.day)
train['pickup_hour'] = train['pickup_datetime'].apply(lambda x:x.hour)
train['pickup_day_of_week'] = train['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
train = pd.concat([train, pd.get_dummies(pd.Series(train['pickup_day_of_week']))], axis = 1)
train['weekend'] = np.where((train['Saturday'] == 1) | (train['Sunday'] == 1), 1, 0)
train['rush_hour'] = np.where((train['pickup_hour'] >= 16) & (train['pickup_hour'] < 20) & (train['weekend'] == 0), 1, 0)
train['overnight'] = np.where((train['pickup_hour'] < 6) | (train['pickup_hour'] > 20), 1, 0)
train['holiday'] = np.where((train['pickup_day'] == 1) | (train['pickup_day'] == 21), 1, 0)

## Airport
train['pickup_LGA'] = train.apply(lambda row:isAirport(row['pickup_latitude'], row['pickup_longitude'], 'LGA'), axis = 1)
train['dropoff_LGA'] = train.apply(lambda row:isAirport(row['dropoff_latitude'], row['dropoff_longitude'], 'LGA'), axis = 1)
train['LGA'] = np.where(train['pickup_LGA'] + train['dropoff_LGA'] > 0, 1, 0)
train['pickup_JFK'] = train.apply(lambda row:isAirport(row['pickup_latitude'], row['pickup_longitude'], 'JFK'), axis = 1)
train['dropoff_JFK'] = train.apply(lambda row:isAirport(row['dropoff_latitude'], row['dropoff_longitude'], 'JFK'), axis = 1)
train['JFK'] = np.where(train['pickup_JFK'] + train['dropoff_JFK'] > 0, 1, 0)
train['pickup_EWR'] = train.apply(lambda row:isAirport(row['pickup_latitude'], row['pickup_longitude'], 'EWR'), axis = 1)
train['dropoff_EWR'] = train.apply(lambda row:isAirport(row['dropoff_latitude'], row['dropoff_longitude'], 'EWR'), axis = 1)
train['EWR'] = np.where(train['pickup_EWR'] + train['dropoff_EWR'] > 0, 1, 0)
train['pickup_airport'] = np.where(train['pickup_LGA'] + train['pickup_JFK'] + train['pickup_EWR'] > 0, 1, 0)
train['dropoff_airport'] = np.where(train['dropoff_LGA'] + train['dropoff_JFK'] + train['dropoff_EWR'] > 0, 1, 0)
train['airport'] = np.where(train['LGA'] + train['JFK'] + train['EWR'] > 0, 1, 0)

## Borough
train['pickup_borough'] = train.apply(lambda row:getBorough(row['pickup_latitude'], row['pickup_longitude']), axis = 1)
train['dropoff_borough'] = train.apply(lambda row:getBorough(row['dropoff_latitude'], row['dropoff_longitude']), axis = 1)
train['change_borough'] = np.where(train['pickup_borough'] != train['dropoff_borough'], 1, 0)
train = pd.concat([train, pd.get_dummies(pd.Series(train['pickup_borough']), prefix = 'pickup', prefix_sep = '_')], axis = 1)
train = pd.concat([train, pd.get_dummies(pd.Series(train['dropoff_borough']), prefix = 'dropoff', prefix_sep = '_')], axis = 1)

## Haversine distance
train['haversine'] = np.nan
for i in range(0, train.shape[0]):
    pickup = (train.at[i, 'pickup_latitude'], train.at[i, 'pickup_longitude'])
    dropoff = (train.at[i, 'dropoff_latitude'], train.at[i, 'dropoff_longitude'])
    train.at[i, 'haversine'] = haversine(pickup, dropoff, unit = 'mi')
    
## Selecting variables of interest
train = train[['fare_amount', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'distance', 'haversine', 
               'duration', 'passenger_count', 'pickup_day', 'holiday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',  'Friday', 
               'Saturday','weekend', 'pickup_hour', 'rush_hour', 'overnight', 'airport', 'LGA', 'JFK', 'EWR', 'pickup_LGA', 'dropoff_LGA', 
               'pickup_JFK', 'dropoff_JFK', 'pickup_EWR', 'dropoff_EWR', 'pickup_airport', 'dropoff_airport', 'change_borough', 
               'pickup_bronx', 'pickup_brooklyn', 'pickup_manhattan', 'pickup_queens', 'pickup_staten_island', 'dropoff_bronx', 
               'dropoff_brooklyn', 'dropoff_manhattan','dropoff_queens', 'dropoff_staten_island']]

In [5]:
## Date/time
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'], format = '%Y-%m-%d %H:%M:%S UTC')
test['pickup_date'] = test['pickup_datetime'].dt.date
test['pickup_day'] = test['pickup_datetime'].apply(lambda x:x.day)
test['pickup_hour'] = test['pickup_datetime'].apply(lambda x:x.hour)
test['pickup_day_of_week'] = test['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
test = pd.concat([test, pd.get_dummies(pd.Series(test['pickup_day_of_week']))], axis = 1)
test['weekend'] = np.where((test['Saturday'] == 1) | (test['Sunday'] == 1), 1, 0)
test['rush_hour'] = np.where((test['pickup_hour'] >= 16) & (test['pickup_hour'] < 20) & (test['weekend'] == 0), 1, 0)
test['overnight'] = np.where((test['pickup_hour'] < 6) | (test['pickup_hour'] > 20), 1, 0)
test['holiday'] = np.where((test['pickup_day'] == 1) | (test['pickup_day'] == 21), 1, 0)

## Airport
test['pickup_LGA'] = test.apply(lambda row:isAirport(row['pickup_latitude'], row['pickup_longitude'], 'LGA'), axis = 1)
test['dropoff_LGA'] = test.apply(lambda row:isAirport(row['dropoff_latitude'], row['dropoff_longitude'], 'LGA'), axis = 1)
test['LGA'] = np.where(test['pickup_LGA'] + test['dropoff_LGA'] > 0, 1, 0)
test['pickup_JFK'] = test.apply(lambda row:isAirport(row['pickup_latitude'], row['pickup_longitude'], 'JFK'), axis = 1)
test['dropoff_JFK'] = test.apply(lambda row:isAirport(row['dropoff_latitude'], row['dropoff_longitude'], 'JFK'), axis = 1)
test['JFK'] = np.where(test['pickup_JFK'] + test['dropoff_JFK'] > 0, 1, 0)
test['pickup_EWR'] = test.apply(lambda row:isAirport(row['pickup_latitude'], row['pickup_longitude'], 'EWR'), axis = 1)
test['dropoff_EWR'] = test.apply(lambda row:isAirport(row['dropoff_latitude'], row['dropoff_longitude'], 'EWR'), axis = 1)
test['EWR'] = np.where(test['pickup_EWR'] + test['dropoff_EWR'] > 0, 1, 0)
test['pickup_airport'] = np.where(test['pickup_LGA'] + test['pickup_JFK'] + test['pickup_EWR'] > 0, 1, 0)
test['dropoff_airport'] = np.where(test['dropoff_LGA'] + test['dropoff_JFK'] + test['dropoff_EWR'] > 0, 1, 0)
test['airport'] = np.where(test['LGA'] + test['JFK'] + test['EWR'] > 0, 1, 0)

## Borough
test['pickup_borough'] = test.apply(lambda row:getBorough(row['pickup_latitude'], row['pickup_longitude']), axis = 1)
test['dropoff_borough'] = test.apply(lambda row:getBorough(row['dropoff_latitude'], row['dropoff_longitude']), axis = 1)
test['change_borough'] = np.where(test['pickup_borough'] != test['dropoff_borough'], 1, 0)
test = pd.concat([test, pd.get_dummies(pd.Series(test['pickup_borough']), prefix = 'pickup', prefix_sep = '_')], axis = 1)
test = pd.concat([test, pd.get_dummies(pd.Series(test['dropoff_borough']), prefix = 'dropoff', prefix_sep = '_')], axis = 1)

## Haversine distance
test['haversine'] = np.nan
for i in range(0, test.shape[0]):
    pickup = (test.at[i, 'pickup_latitude'], test.at[i, 'pickup_longitude'])
    dropoff = (test.at[i, 'dropoff_latitude'], test.at[i, 'dropoff_longitude'])
    test.at[i, 'haversine'] = haversine(pickup, dropoff, unit = 'mi')

## Selecting variables of interest
test = test[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'distance', 'haversine', 
               'duration', 'passenger_count', 'pickup_day', 'holiday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',  'Friday', 
               'Saturday','weekend', 'pickup_hour', 'rush_hour', 'overnight', 'airport', 'LGA', 'JFK', 'EWR', 'pickup_LGA', 'dropoff_LGA', 
               'pickup_JFK', 'dropoff_JFK', 'pickup_EWR', 'dropoff_EWR', 'pickup_airport', 'dropoff_airport', 'change_borough', 
               'pickup_bronx', 'pickup_brooklyn', 'pickup_manhattan', 'pickup_queens', 'pickup_staten_island', 'dropoff_bronx', 
               'dropoff_brooklyn', 'dropoff_manhattan','dropoff_queens', 'dropoff_staten_island']]

In [6]:
## Writing training and testing sets to csv files
train.to_csv('Data/W23P1_train_final.csv', index = False)
test.to_csv('Data/W23P1_test_final.csv', index = False)